### LlamaIndexの準備

In [ ]:
# パッケージのインストール
!pip install llama-index==0.10.39
!pip install llama-index-llms-gemini
!pip install llama-index-embeddings-huggingface

In [20]:
import google.generativeai as genai
from dotenv import load_dotenv
import os
from tqdm.notebook import tqdm

load_dotenv()
GOOGLE_API_KEY=os.environ.get("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

In [21]:
import logging
import sys

# ログレベルの設定
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG, force=True)

In [22]:
from llama_index.core import Settings
from llama_index.llms.gemini import Gemini
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# LLMの準備
Settings.llm = Gemini(
    model_name="models/gemini-1.0-pro",
    safety_settings={
        "HARM_CATEGORY_HARASSMENT": "BLOCK_NONE",
        "HARM_CATEGORY_HATE_SPEECH": "BLOCK_NONE",
        "HARM_CATEGORY_SEXUALLY_EXPLICIT" : "BLOCK_NONE",
        "HARM_CATEGORY_DANGEROUS_CONTENT" : "BLOCK_NONE"
    }
)

# 埋め込みモデルの準備
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-m3"
)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-m3
DEBUG:urllib3.connectionpool:Resetting dropped connection: huggingface.co
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /BAAI/bge-m3/resolve/main/modules.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /BAAI/bge-m3/resolve/main/config_sentence_transformers.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /BAAI/bge-m3/resolve/main/README.md HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /BAAI/bge-m3/resolve/main/modules.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /BAAI/bge-m3/resolve/main/sentence_bert_config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /BAAI/bge-m3/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /BAAI/bge-m3/resolve/main/model.safe

### 質問コンテキストデータセットの生成

In [23]:
# Colabでのasyncio利用の有効化
import nest_asyncio
nest_asyncio.apply()

In [24]:
from llama_index.core import SimpleDirectoryReader

# ドキュメントの読み込み (dataフォルダにドキュメントを配置しておきます)
documents = SimpleDirectoryReader("data").load_data()

DEBUG:llama_index.core.readers.file.base:> [SimpleDirectoryReader] Total files added: 7
DEBUG:fsspec.local:open file: /work/gemini_npaka/llamaindex/data/akazukin1.txt
DEBUG:fsspec.local:open file: /work/gemini_npaka/llamaindex/data/akazukin2.txt
DEBUG:fsspec.local:open file: /work/gemini_npaka/llamaindex/data/akazukin3.txt
DEBUG:fsspec.local:open file: /work/gemini_npaka/llamaindex/data/akazukin4.txt
DEBUG:fsspec.local:open file: /work/gemini_npaka/llamaindex/data/akazukin5.txt
DEBUG:fsspec.local:open file: /work/gemini_npaka/llamaindex/data/akazukin6.txt
DEBUG:fsspec.local:open file: /work/gemini_npaka/llamaindex/data/akazukin7.txt


In [25]:
from llama_index.core.node_parser import SentenceSplitter

# ドキュメントからノードを取得
node_parser = SentenceSplitter()
nodes = node_parser.get_nodes_from_documents(documents)

# ノードIDを手動で設定
for idx, node in enumerate(nodes):
    node.id_ = f"node_{idx}"

DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 第1章：ネオン街の影
2078年、東京。ネオン街の闇に紛れ、少女アズキはフードを深く被り、路地...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 第2章：牢獄の出会い
暗い牢獄に閉じ込められたアズキ。絶望に打ちひしがれながらも、彼女は諦めな...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 第3章：情報収集
ネオン街の裏社会に潜伏するアズキとオオカミ。彼らは帝国の秘密情報を収集するた...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 第4章：裏切り
ある日、オオカミの裏切りが発覚する。彼は帝国に情報を売っていたのだ。
アズキは...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 第5章：決戦
ついに、アズキとオオカミは帝国の本拠地へ潜入する。そこで、彼らは帝国の恐るべき計...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 第6章：希望の光
激しい戦いの末、アズキとオオカミはシステムを破壊することに成功する。帝国は崩...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 第7章：新たな旅
街に平和が訪れた数年後、アズキは新たな旅に出る。
彼女は世界中を旅し、他の抑...


In [26]:
nodes

[TextNode(id_='node_0', embedding=None, metadata={'file_path': '/work/gemini_npaka/llamaindex/data/akazukin1.txt', 'file_name': 'akazukin1.txt', 'file_type': 'text/plain', 'file_size': 632, 'creation_date': '2024-11-09', 'last_modified_date': '2024-11-09'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='cd1e6db3-b625-43fc-a298-e0d1f725db63', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': '/work/gemini_npaka/llamaindex/data/akazukin1.txt', 'file_name': 'akazukin1.txt', 'file_type': 'text/plain', 'file_size': 632, 'creation_date': '2024-11-09', 'last_modified_date': '2024-11-09'}, hash='389805e4071c5b62d98f4afd2ae515fdd3e09b766b199fc0077175dfd18deac6')}, text='第1章：ネオン街の影\n2078年、東京。ネオン街の闇に紛れ、少女ア

In [27]:
from llama_index.core.evaluation import generate_question_context_pairs
from llama_index.llms.openai import OpenAI

# 日本語テンプレートの準備
DEFAULT_QA_GENERATE_PROMPT_TMPL = """コンテキスト情報は以下のとおりです。

---------------------
{context_str}
---------------------

予備知識がなく、文脈情報が与えられた場合。
以下のクエリに基づいて問題のみを生成します。

あなたは先生です。
あなたのタスクは、今後の試験用に {num_questions_per_chunk} 個の質問を作成することです。
問題は文書全体にわたって多様でなければなりません。
設問は提供されたコンテキスト情報に限定してください。
質問 1や問題1などは出力せずに作成した質問のみ出力してください"""

# 質問コンテキストデータセットの生成
qa_dataset = generate_question_context_pairs(
    nodes,
    llm=Settings.llm,
    num_questions_per_chunk=2,
    qa_generate_prompt_tmpl=DEFAULT_QA_GENERATE_PROMPT_TMPL,
)

100%|██████████| 7/7 [00:09<00:00,  1.33s/it]


In [28]:
qa_dataset

EmbeddingQAFinetuneDataset(queries={'bde1678c-9b5e-40e7-848d-f1269a868c6d': 'アズキがハッカー集団「紅ずきん」の一員である理由は？', '92027deb-8160-4bf0-ba16-a4714ca0fec5': 'アズキが帝国の警備隊に捕らえられた状況を説明せよ。', 'e03778d0-3eef-4e9c-ba07-cd4befdb7cf1': 'アズキとオオカミが出会った状況を説明してください。', '78852c97-a098-471a-97fb-7a889d06955b': 'アズキとオオカミが協力して達成したことを述べてください。', '0c49a378-ee3a-4837-8e4f-a2aa837697c9': 'アズキとオオカミが情報収集のために実行したハッキング作戦について説明してください。', '1add70b3-06eb-4401-9cba-096673eb3ef4': '帝国がアズキとオオカミを追跡するために仕かけた罠を挙げてください。', '241ccb92-842f-408f-9f60-b805b1058ba2': 'オオカミが帝国に情報を売っていた理由は？', '1e377ac6-c437-46c0-bce6-28cca0002be7': 'アズキがオオカミを信じることにした理由は？', '00bb6342-5415-4188-836c-c6cf72f43599': '帝国が開発していた洗脳システムの目的は何でしたか？', '6638a831-a363-45b9-9053-4bdd129be841': 'アズキとオオカミが帝国の本拠地に潜入した理由は？', 'efe35741-64b1-46eb-a375-741932a9780a': 'アズキとオオカミがシステムを破壊した結果、街にどのような変化が生じたか。', 'b2990894-7e35-49ed-b78b-6ec3df13addd': 'オオカミの死後、アズキが下した決意とは何か。', '905d3982-1c74-4788-8af9-59874439e603': 'アズキの旅の目的は何ですか？', '4a8caf9f-3499-4fe7-b8df-be81ab9da0cb': 'アズキの出身地はどこか

In [29]:
# 質問コンテキストデータセットの保存
qa_dataset.save_json("pg_eval_dataset.json")

In [30]:
from llama_index.core.evaluation import EmbeddingQAFinetuneDataset

# 質問コンテキストデータセットの読み込み
qa_dataset = EmbeddingQAFinetuneDataset.from_json("pg_eval_dataset.json")

In [31]:
# 生成したデータセットの確認 (先頭5つ)
for key in list(qa_dataset.queries.keys())[:5]:
    print("queries", qa_dataset.queries[key])
    print("relevant_docs", qa_dataset.relevant_docs[key])

queries アズキがハッカー集団「紅ずきん」の一員である理由は？
relevant_docs ['node_0']
queries アズキが帝国の警備隊に捕らえられた状況を説明せよ。
relevant_docs ['node_0']
queries アズキとオオカミが出会った状況を説明してください。
relevant_docs ['node_1']
queries アズキとオオカミが協力して達成したことを述べてください。
relevant_docs ['node_1']
queries アズキとオオカミが情報収集のために実行したハッキング作戦について説明してください。
relevant_docs ['node_2']


### Retrieval Evaluation (取得評価)

In [16]:
from llama_index.core import VectorStoreIndex

# インデックスとRetrieverの準備
vector_index = VectorStoreIndex(nodes)
retriever = vector_index.as_retriever(similarity_top_k=3)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
from llama_index.core.evaluation import RetrieverEvaluator

# RetrieverEvaluatorの準備
retriever_evaluator = RetrieverEvaluator.from_metric_names(
    ["mrr", "hit_rate"],
    retriever=retriever
)

In [32]:
mrr_values = []
hit_rate_values = []

# 評価の実行
for key in list(qa_dataset.queries.keys()):
    # 1クエリの評価の実行
    result =retriever_evaluator.evaluate(
        query=qa_dataset.queries[key],
        expected_ids=qa_dataset.relevant_docs[key]
    )

    # 集計
    mrr_values.append(result.metric_dict["mrr"].score)
    hit_rate_values.append(result.metric_dict["hit_rate"].score)

    # 確認
    print(result)

# 確認
mmr_average = sum(mrr_values) / len(mrr_values)
hit_rate_average = sum(hit_rate_values) / len(hit_rate_values)
print("MRR:", mmr_average, "Hit Rate:", hit_rate_average)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

DEBUG:llama_index.core.indices.utils:> Top 3 nodes:
> [Node node_0] [Similarity score:             0.637423] 第1章：ネオン街の影
2078年、東京。ネオン街の闇に紛れ、少女アズキはフードを深く被り、路地裏を疾走していた。彼女は16歳、ハッカー集団「紅ずきん」の一員だ。
アズキのターゲットは、企業連合「...
> [Node node_1] [Similarity score:             0.541058] 第2章：牢獄の出会い
暗い牢獄に閉じ込められたアズキ。絶望に打ちひしがれながらも、彼女は諦めなかった。脱出する方法を探るアズキの前に、謎めいた青年が現れる。
青年はハッカー「オオカミ」と呼ばれ、...
> [Node node_2] [Similarity score:             0.507502] 第3章：情報収集
ネオン街の裏社会に潜伏するアズキとオオカミ。彼らは帝国の秘密情報を収集するために、様々なハッキング作戦を実行していく。
しかし、帝国も黙っていない。アズキたちを追跡し、次々と罠...
Query: アズキがハッカー集団「紅ずきん」の一員である理由は？
Metrics: {'mrr': 1.0, 'hit_rate': 1.0}



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

DEBUG:llama_index.core.indices.utils:> Top 3 nodes:
> [Node node_0] [Similarity score:             0.590785] 第1章：ネオン街の影
2078年、東京。ネオン街の闇に紛れ、少女アズキはフードを深く被り、路地裏を疾走していた。彼女は16歳、ハッカー集団「紅ずきん」の一員だ。
アズキのターゲットは、企業連合「...
> [Node node_3] [Similarity score:             0.579266] 第4章：裏切り
ある日、オオカミの裏切りが発覚する。彼は帝国に情報を売っていたのだ。
アズキはショックを受け、オオカミを問い詰める。オオカミは、妹を人質に取られており、帝国に協力せざるを得なかっ...
> [Node node_2] [Similarity score:             0.572892] 第3章：情報収集
ネオン街の裏社会に潜伏するアズキとオオカミ。彼らは帝国の秘密情報を収集するために、様々なハッキング作戦を実行していく。
しかし、帝国も黙っていない。アズキたちを追跡し、次々と罠...
Query: アズキが帝国の警備隊に捕らえられた状況を説明せよ。
Metrics: {'mrr': 1.0, 'hit_rate': 1.0}



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

DEBUG:llama_index.core.indices.utils:> Top 3 nodes:
> [Node node_5] [Similarity score:             0.585484] 第6章：希望の光
激しい戦いの末、アズキとオオカミはシステムを破壊することに成功する。帝国は崩壊し、街は解放される。
しかし、オオカミは戦いの傷が深すぎ、命を落としてしまう。
アズキはオオカミの...
> [Node node_3] [Similarity score:             0.516703] 第4章：裏切り
ある日、オオカミの裏切りが発覚する。彼は帝国に情報を売っていたのだ。
アズキはショックを受け、オオカミを問い詰める。オオカミは、妹を人質に取られており、帝国に協力せざるを得なかっ...
> [Node node_1] [Similarity score:             0.50772] 第2章：牢獄の出会い
暗い牢獄に閉じ込められたアズキ。絶望に打ちひしがれながらも、彼女は諦めなかった。脱出する方法を探るアズキの前に、謎めいた青年が現れる。
青年はハッカー「オオカミ」と呼ばれ、...
Query: オオカミの死後、アズキが下した決意とは何か。
Metrics: {'mrr': 1.0, 'hit_rate': 1.0}



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

DEBUG:llama_index.core.indices.utils:> Top 3 nodes:
> [Node node_6] [Similarity score:             0.556335] 第7章：新たな旅
街に平和が訪れた数年後、アズキは新たな旅に出る。
彼女は世界中を旅し、他の抑圧された人々を助け、自由と正義のために戦い続ける。
アズキは、ネオン街の影から生まれ、希望の光となっ...
> [Node node_1] [Similarity score:             0.475553] 第2章：牢獄の出会い
暗い牢獄に閉じ込められたアズキ。絶望に打ちひしがれながらも、彼女は諦めなかった。脱出する方法を探るアズキの前に、謎めいた青年が現れる。
青年はハッカー「オオカミ」と呼ばれ、...
> [Node node_0] [Similarity score:             0.475018] 第1章：ネオン街の影
2078年、東京。ネオン街の闇に紛れ、少女アズキはフードを深く被り、路地裏を疾走していた。彼女は16歳、ハッカー集団「紅ずきん」の一員だ。
アズキのターゲットは、企業連合「...
Query: アズキの旅の目的は何ですか？
Metrics: {'mrr': 1.0, 'hit_rate': 1.0}



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

DEBUG:llama_index.core.indices.utils:> Top 3 nodes:
> [Node node_6] [Similarity score:             0.551354] 第7章：新たな旅
街に平和が訪れた数年後、アズキは新たな旅に出る。
彼女は世界中を旅し、他の抑圧された人々を助け、自由と正義のために戦い続ける。
アズキは、ネオン街の影から生まれ、希望の光となっ...
> [Node node_0] [Similarity score:             0.470442] 第1章：ネオン街の影
2078年、東京。ネオン街の闇に紛れ、少女アズキはフードを深く被り、路地裏を疾走していた。彼女は16歳、ハッカー集団「紅ずきん」の一員だ。
アズキのターゲットは、企業連合「...
> [Node node_5] [Similarity score:             0.460826] 第6章：希望の光
激しい戦いの末、アズキとオオカミはシステムを破壊することに成功する。帝国は崩壊し、街は解放される。
しかし、オオカミは戦いの傷が深すぎ、命を落としてしまう。
アズキはオオカミの...
Query: アズキの出身地はどこか、そして彼女はどのようにして希望の象徴となったのですか？
Metrics: {'mrr': 1.0, 'hit_rate': 1.0}

MRR: 0.9642857142857143 Hit Rate: 1.0


### Response Evaluation (応答評価)

In [33]:
# LLMの準備
Settings.llm = Gemini(
    model_name="models/gemini-1.0-pro",
    safety_settings={
        "HARM_CATEGORY_HARASSMENT": "BLOCK_NONE",
        "HARM_CATEGORY_HATE_SPEECH": "BLOCK_NONE",
        "HARM_CATEGORY_SEXUALLY_EXPLICIT" : "BLOCK_NONE",
        "HARM_CATEGORY_DANGEROUS_CONTENT" : "BLOCK_NONE"
    }
)

In [ ]:
# 環境変数の準備 (左端の鍵アイコンでOPENAI_API_KEYを設定)
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [ ]:
from llama_index.core.prompts import PromptTemplate
from llama_index.core.evaluation import FaithfulnessEvaluator
from llama_index.llms.openai import OpenAI

# 日本語テンプレートの準備
DEFAULT_EVAL_TEMPLATE = PromptTemplate(
    """特定の情報がコンテキストによってサポートされているかどうかを教えてください。
YES か NO で答える必要があります。
コンテキストの大部分が無関係であっても、コンテキストのいずれかが情報をサポートしている場合は、YES と答えてください。いくつかの例を以下に示します。

Information: アップルパイは通常、二重の生地です。
Context: アップルパイは、主な詰め物の材料がリンゴであるフルーツパイです。
アップルパイには、ホイップクリーム、アイスクリーム、カスタード、チェダーチーズが添えられることがよくあります。
通常は二重の生地で、中身の上下にペストリーが入っています。上部の生地は固体または格子状の場合があります。
Answer: YES

Information: アップルパイは不味い。
Context: アップルパイは、主な詰め物の材料がリンゴであるフルーツパイです。
アップルパイには、ホイップクリーム、アイスクリーム、カスタード、チェダーチーズが添えられることがよくあります。
通常は二重の生地で、中身の上下にペストリーが入っています。上部の生地は固体または格子状の場合があります。
Answer: NO

Information: {query_str}
Context: {context_str}
Answer: """
)

# FaithfulnessEvaluatorの準備
evaluator = FaithfulnessEvaluator(
    llm=OpenAI(model="gpt-4-turbo"),
    eval_template=DEFAULT_EVAL_TEMPLATE
)

In [ ]:
from llama_index.core import VectorStoreIndex

# インデックスとクエリエンジンの準備
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

In [ ]:
faithfulness_values = []

# 評価の実行
for key in list(qa_dataset.queries.keys()):
    # 1クエリの評価の実行
    response = query_engine.query(qa_dataset.queries[key])
    result = evaluator.evaluate_response(response=response)

    # 集計
    faithfulness_values.append(1 if result.passing else 0)

    # 確認
    print("Information:", qa_dataset.queries[key])
    print("Context:")
    for node in response.source_nodes:
        print(node.get_text())
    print("Answer:", result.passing)

# 確認
faithfulness_average = sum(faithfulness_values) / len(faithfulness_values)
print("Faithfulness:", faithfulness_average)